In [88]:
import pandas as pd
import  numpy as np
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [89]:
!pip  install difflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


###Data collectiona nd preporocessing

In [90]:
#loading data from the data frame

movies_data = pd.read_csv('/content/movies.csv')
movies_data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [91]:
#printing the first 5 rows of the movie
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [92]:
#total num of rows and columns
movies_data.shape

(4803, 24)

In [93]:
#selecting the relevant features
#contant based and popularity based recomendation system

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [94]:
#replacing the null values
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna(' ')

In [95]:
#combining all the files selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+" "+movies_data['tagline']+" "+movies_data['cast']+" "+movies_data['director']

In [96]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance   A newlywed couple's honeymoon...
4800    Comedy Drama Romance TV Movie date love at fir...
4801        A New Yorker in Shanghai Daniel Henney Eli...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [97]:
#we are going to converting text data to feature vectors
vectorizer = TfidfVectorizer()

In [98]:
feature_vectorizer = vectorizer.fit_transform(combined_features)

In [99]:
print(feature_vectorizer)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

###getting similarity score using cosine similarity function

In [100]:
similarity = cosine_similarity(feature_vectorizer)

In [101]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [102]:
similarity_values = list(similarity)

In [103]:
#getting the movie name from the user
movie_name = input("enter your favorite movie name: ")

enter your favorite movie name: The Last Airbender


In [104]:
#creatiing a list with all the movies given in the dataset
list_of_titles = movies_data['title'].tolist()
print(list_of_titles)



['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [105]:
#finding the close movie name given from the user
find_close_match = difflib.get_close_matches(movie_name,list_of_titles)
print(find_close_match)

['The Last Airbender', 'The Last Sin Eater', 'The Last Five Years']


In [106]:
close_match = find_close_match[0]
print(close_match)

The Last Airbender


In [107]:
#finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title==close_match]['index'].values[0]
print(index_of_the_movie)


138


In [108]:
#getting  list of similarity movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.04043926401367979), (1, 0.04085748517694469), (2, 0.02791394610368834), (3, 0.006578392460998658), (4, 0.013294129344076304), (5, 0.040453690453654066), (6, 0.016991499295725212), (7, 0.013780470894162514), (8, 0.029493773499584562), (9, 0.02506856763617156), (10, 0.02859222192748208), (11, 0.01212819938266426), (12, 0.025955639141410813), (13, 0.012585832062626177), (14, 0.02525486679324467), (15, 0.027263266143435334), (16, 0.013066815721621484), (17, 0.024829582058925027), (18, 0.005628958731776058), (19, 0.043752303065051165), (20, 0.02531500431533998), (21, 0.011457830993980046), (22, 0.04024569953800697), (23, 0.016886791366980337), (24, 0.03649509191884789), (25, 0.04190695561991919), (26, 0.045514455580932145), (27, 0.012436787092948667), (28, 0.013654124566543201), (29, 0.01428424741072931), (30, 0.039357446159130435), (31, 0.028176651068663645), (32, 0.0373063156475255), (33, 0.013694848697242386), (34, 0.047678756197162164), (35, 0.014376164808619415), (36, 0.15216129

In [109]:
len(similarity_score)

4803

In [110]:
#sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)
print(sorted_similar_movies)

[(138, 0.9999999999999997), (36, 0.15216129788920735), (3025, 0.14694904256888067), (2666, 0.12457881240013022), (68, 0.11846458075866884), (514, 0.11357204450341589), (746, 0.11291412588239365), (326, 0.10704198524694081), (2993, 0.10701142126317659), (3867, 0.10671062832547623), (628, 0.10593913383261402), (2186, 0.10252770239187545), (193, 0.09857092285435923), (1406, 0.09734184480557825), (855, 0.09514366244335369), (545, 0.09390081773333156), (3670, 0.09277946978572502), (195, 0.09264313823576964), (1008, 0.08583908106754319), (348, 0.08307559066109294), (92, 0.08102717941602579), (1765, 0.0808690817314183), (3653, 0.08020156664423844), (562, 0.07938371441719018), (1290, 0.07933959258123391), (576, 0.07861742340249459), (564, 0.07826520114138437), (1249, 0.07772084139448435), (1510, 0.07654794848874596), (3177, 0.07481214051942403), (1795, 0.07476416492891892), (785, 0.07474918180208899), (1145, 0.07449917163182), (236, 0.07447356070685661), (769, 0.07409367377091072), (1234, 0.07

In [111]:
#print the name of similar movies based on the index
print("movies_sugessted for you : \n")

i = 1 

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i," ",title_from_index)
    i+=1



movies_sugessted for you : 

1   The Last Airbender
2   Transformers: Age of Extinction
3   The Second Best Exotic Marigold Hotel
4   Slumdog Millionaire
5   Iron Man
6   Ice Age: The Meltdown
7   Reign of Fire
8   Cinderella
9   The Best Exotic Marigold Hotel
10   Human Traffic
11   Saving Private Ryan
12   The Kite Runner
13   After Earth
14   The Nativity Story
15   Gods and Generals
16   Unbreakable
17   Running Forever
18   Night at the Museum: Secret of the Tomb
19   Chappie
20   Ice Age: Dawn of the Dinosaurs
21   How to Train Your Dragon
22   The Newton Boys
23   The Visit
24   The Village
25   Baahubali: The Beginning
26   Lady in the Water
27   Signs
28   Torque
29   1941


In [114]:
#final_block code
movie_name = input("enter your favorite movie name: ")

list_of_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title==close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score,key = lambda x:x[1],reverse = True)


#print the name of similar movies based on the index
print("movies_sugessted for you : \n")

i = 1 

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i," ",title_from_index)
    i+=1



enter your favorite movie name: iron man
movies_sugessted for you : 

1   Iron Man
2   Iron Man 2
3   Iron Man 3
4   Avengers: Age of Ultron
5   The Avengers
6   Captain America: Civil War
7   Captain America: The Winter Soldier
8   Ant-Man
9   X-Men
10   Made
11   X-Men: Apocalypse
12   X2
13   The Incredible Hulk
14   The Helix... Loaded
15   X-Men: First Class
16   X-Men: Days of Future Past
17   Captain America: The First Avenger
18   Kick-Ass 2
19   Guardians of the Galaxy
20   Deadpool
21   Thor: The Dark World
22   G-Force
23   X-Men: The Last Stand
24   Duets
25   Mortdecai
26   The Last Airbender
27   Southland Tales
28   Zathura: A Space Adventure
29   Sky Captain and the World of Tomorrow
